In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

## Naive Bayes (朴素贝叶斯)
前面讲的knn以及decision tree都是属于分类模型，并且此分类模型为hard type，接下来我们讲的朴素贝叶斯也是分类模型，但是是soft type，也就是会计算出各个种类的概率。  
朴素贝叶斯作为贝叶斯定理的分支，我们先从贝叶斯定理讲起。贝叶斯定理基于条件概率从已知判断未知，首先，介绍条件概率的公式：
$$P(A|B) = \frac{P(A)P(B|A)}{P(B)}$$
if B 发生后A的概率如上所示，以现实情况就是，如果被诊断出癌症(B)，那么真的是癌症(A)的概率是多少？首先，你有一个得癌症概率的数据$p_a$，在医生诊断之前，得癌症的概率就是$p_a$，然而，如今被诊断出来，自己得癌症的概率也不代表就是100%，你还有误诊的概率数据$p_b$以及医生诊断病人为癌症的概率$p_c$，显而易见的，$p_a=p_c(1-p_b)+(1-p_c)p_b$，所以被诊断出癌症后，罹癌概率从$p_a$变成了$\frac{p_c\times (1-p_b)}{p_a}$

朴素贝叶斯可以想成是贝叶斯定理里条件概率的B有多个特征，变成是
$$P(A|B,C) = \frac{P(A)P(B|A)P(C|A,B)}{P(B)P(C|B)}$$
朴素贝叶斯里的朴素两个字意思是假设特征之间在统计的意义上是独立的，也就是相关系数为0，所以完整的朴素贝叶斯公式为如下形式：
$$P(A|B,C) = \frac{P(A)P(B|A)P(C|A)}{P(B)*P(C)}$$

### 优缺点

优点：数据少也能训练出不错的结果、可以同时处理多个classes  
缺点：对数据的输入敏感

### 示例代码
我们以判断一句话是否为辱骂性(abusive)的句子为例  
首先，loadDataSet代表产生训练数据，返回训练数据以及标签，createVocabList则是将训练数据取set，记录训练集出现的单词，setOfWords2Vec则是将原始训练数据(单词)转成只有0和1的向量形式

In [2]:
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not
    return postingList,classVec
                 
def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document) #union of the two sets
    return list(vocabSet)

def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

有了句子的向量形式以及标签，我们利用trainNB0进行训练，其输出三个值，分别为各个单词代表非辱骂性的概率、各个单词代表辱骂性的概率、以及整体训练集是辱骂性句子的比例，计算概率的方式就是最直觉的方式 - 出现这个词的次数/这个类里单词的总数  
我们以stupid这个单词为例，在辱骂性的句子里，stupid出现3次，辱骂性句子总共有19个单词，所以概率就是$\frac{3}{19}$  
然而我们知道朴素贝叶斯的公式假设特征之间独立，所以会连续相乘，如果遇到概率为0，不管之后多大相乘都还是0，例如stupid在非辱骂性的概率就是0，所以有个办法是得出概率后加上一个极小值,并且对概率取log方便计算，因为$log(ab)=log(a)+log(b)$,避免之后计算朴素贝叶斯公式时，数值连续相乘导致逼近于0。

In [3]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = np.ones(numWords); p1Num = np.ones(numWords)      #change to ones() 
    p0Denom = 2.0; p1Denom = 2.0                        #change to 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num/p1Denom)          #change to log()
    p0Vect = np.log(p0Num/p0Denom)          #change to log()
    return p0Vect,p1Vect,pAbusive

In [4]:
postingList,classVec = loadDataSet()
vob_list = createVocabList(postingList)
trainMat=[]
for sentence in postingList:
    trainMat.append(setOfWords2Vec(vob_list, sentence))

In [5]:
p0V, p1V, pAb = trainNB0(trainMat, classVec)

In [6]:
for i in range(len(vob_list)):
    print(p0V[i],p1V[i],vob_list[i])

-3.258096538021482 -2.3513752571634776 park
-3.258096538021482 -2.3513752571634776 posting
-2.5649493574615367 -2.3513752571634776 stop
-2.5649493574615367 -3.044522437723423 cute
-2.5649493574615367 -3.044522437723423 problems
-3.258096538021482 -2.3513752571634776 quit
-3.258096538021482 -2.3513752571634776 maybe
-3.258096538021482 -2.3513752571634776 not
-2.5649493574615367 -3.044522437723423 mr
-2.5649493574615367 -2.3513752571634776 to
-2.5649493574615367 -3.044522437723423 how
-2.5649493574615367 -3.044522437723423 ate
-2.5649493574615367 -3.044522437723423 is
-3.258096538021482 -1.9459101490553135 worthless
-2.5649493574615367 -1.9459101490553135 dog
-2.5649493574615367 -3.044522437723423 so
-1.8718021769015913 -3.044522437723423 my
-2.5649493574615367 -3.044522437723423 help
-3.258096538021482 -2.3513752571634776 buying
-3.258096538021482 -2.3513752571634776 garbage
-3.258096538021482 -2.3513752571634776 food
-2.5649493574615367 -3.044522437723423 please
-2.5649493574615367 -3.

接下来就是利用朴素贝叶斯计算各类别的概率了，我们再放一次朴素贝叶斯的公式
$$P(A|B,C) = \frac{P(A)P(B|A)P(C|A)}{P(B)*P(C)}$$
A代表类别，也就是是否为辱骂性的句子，B、C、D...代表特征，这里的特征就是单词，由于我们是计算相对的值来判定类别，所以分母$P(B)*P(C)$可以不用计算，由于我们之前计算的概率取了log，所以分子相乘的运算就是log相加的运算

In [7]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)    #element-wise mult
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB0(np.array(trainMat),np.array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry = ['stupid', 'garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))

In [8]:
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


以上的例子，我们只对词做出现或缺失的操作，接下来介绍的是bag-of-words，将词语的频率也考虑进去，我们以分辨垃圾邮件为例

In [9]:
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec


In [11]:
import re
regEx=re.compile('\\W*')
emailText = open('./datasets/email/ham/6.txt').read()
listOfTokens=regEx.split(emailText)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: FutureWarning: split() requires a non-empty pattern match.
  after removing the cwd from sys.path.


In [12]:
import random
def textParse(bigString):    #input is big string, #output is word list
    import re
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2] 
    
def spamTest():
    docList=[]; classList = []; fullText =[]
    for i in range(1,26):
        wordList = textParse(open('./datasets/email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('./datasets/email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    trainingSet = list(range(50)); testSet=[]           #create test set
    for i in range(10):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del trainingSet[randIndex]
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(np.array(trainMat),np.array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
            print("classification error",docList[docIndex])
    print('the error rate is: ',float(errorCount)/len(testSet))
    #return vocabList,fullText
spamTest()

classification error ['yay', 'you', 'both', 'doing', 'fine', 'working', 'mba', 'design', 'strategy', 'cca', 'top', 'art', 'school', 'new', 'program', 'focusing', 'more', 'right', 'brained', 'creative', 'and', 'strategic', 'approach', 'management', 'the', 'way', 'done', 'today']
the error rate is:  0.1


/usr/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
